---
**Observation:**  

1. Null values are present in the following columns:
   - **`videoDefaultLanguage`**  (will be dropped after data cleaning)
   - **`videoDefaultAudioLanguage`** 
   - **`channelCountry`**

---

2. The following columns will be dropped as part of data cleaning:
   - **`videoDescription`**: Reserved for analysis in future NLP project with a broader dataset.  
   - **`videoLiveBroadcastContent`**: All values are `'none'`, providing no variability or insights. 
   - **`videoFavoriteCount`**: All values are `0`, making it redundant.  
   - **`videoTags`**: Reserved for analysis in future NLP project with a broader dataset.  
   - **`videoUniqueId`**: Identified as a duplicate column.  
   - **`channelIdUnique`**: Identified as a duplicate column.  
   - **`channelTitleCheck`**: Identified as a duplicate column.  
   - **`channelDescription`**: Reserved for analysis in future NLP project with a broader dataset.
---

3. The columns **`channelName`** and **`videoTitle`** require further processing due to the presence of:
    - Multilingual text.  
    - Emojis and special characters.  

---

In [1]:
def Source_File_Extraction(repo_url, kaggle_repo_url, source_path):
    """
    This function checks if a specified Git repository already exists in the local system.
    If the repository exists, it pulls the latest changes from the remote repository.
    If the repository doesn't exist, it clones the repository from the provided URL.
    
    After ensuring the repository is up-to-date, it searches for a JSON file that starts with "S_" 
    and ends with "records.json" in the specified source directory, loads the file using pandas, 
    and returns the data as a DataFrame.

    Args:
    - repo_url (str): The URL of the Git repository to clone if not already present.
    - kaggle_repo_url (str): The local path where the repository is stored or will be cloned to.
    - source_path (str): The directory where the JSON file is stored.

    Returns:
    - pd.DataFrame: The data from the JSON file as a pandas DataFrame.
    """
    if os.path.exists(kaggle_repo_url):
        print("Already cloned and the repo file exists")
        repo = git.Repo(kaggle_repo_url)  # Access the existing repository
        origin = repo.remote(name='origin')  # Get the remote repository
        origin.pull()  # Pull the latest changes from the repository
        print("Successfully pulled the git repo before push")
    else:
        # Clone the repository if it doesn't exist
        repo = git.Repo.clone_from(repo_url, kaggle_repo_url)
        print("Successfully cloned the git repo")
    
    # List all files in the source path and find the relevant JSON file
    output_files = os.listdir(source_path)
    Source_File = max([i for i in output_files if i.startswith("S_") and i.endswith('records.json')])
    
    # Read the found JSON file into a pandas DataFrame
    Source_File = pd.read_json(f'{source_path}/{Source_File}')
    
    return Source_File

In [2]:
def Requirement_File_Extraction(repo_url, kaggle_repo_url, requirement_path):
    """
    Ensures the repository is up-to-date by either pulling the latest changes or cloning it.
    Then, extracts and returns the most recent JSON file starting with "RE_" and ending with 
    "country_details.json" from the specified requirement directory as a pandas DataFrame.

    Args:
    - repo_url (str): Git repository URL to clone if not present.
    - kaggle_repo_url (str): Local directory path for the repository.
    - requirement_path (str): Directory containing the JSON files.

    Returns:
    - pd.DataFrame: Data from the most recent JSON file.
    """
    
    # Check if the repository already exists locally
    if os.path.exists(kaggle_repo_url):
        print("Already cloned and the repo file exists")
        
        # Access the existing repository and pull the latest changes
        repo = git.Repo(kaggle_repo_url)
        origin = repo.remote(name='origin')
        origin.pull()  # Pull the latest changes
        print("Successfully pulled the git repo before push")
    else:
        # Clone the repository if it doesn't exist locally
        repo = git.Repo.clone_from(repo_url, kaggle_repo_url)
        print("Successfully cloned the git repo")
    
    # List all files in the requirement directory
    output_files = os.listdir(requirement_path)
    
    # Find the most recent JSON file that starts with "RE_" and ends with "country_details.json"
    Requirement_File = max([i for i in output_files if i.startswith("RE_") and i.endswith('country_details.json')])
    
    # Read the found JSON file into a pandas DataFrame
    Requirement_File = pd.read_json(f'{requirement_path}/{Requirement_File}')
    
    return Requirement_File

In [3]:
def DataCleaning(Target_File):
    """
    Cleans the input DataFrame by performing the following operations:
    1. Drops irrelevant columns.
    2. Removes duplicate rows.
    3. Filters videos based on language (only those with 'videoDefaultAudioLanguage' starting with 'en').
    4. Translates non-ASCII characters in 'channelName' and 'videoTitle' to English.
    5. Removes emojis and decodes HTML entities from 'channelName' and 'videoTitle'.
    6. Removes non-ASCII characters from 'channelName' and 'videoTitle'.
    7. Fills missing values in 'channelCountry' with 'Unknown'.
    8. Returns the cleaned DataFrame.

    Args:
    - Target_File (pd.DataFrame): The DataFrame to clean.

    Returns:
    - pd.DataFrame: The cleaned DataFrame.
    """
    
    # Drop irrelevant columns
    Target_File = Target_File.drop(['videoDescription', 'videoLiveBroadcastContent', 'videoFavoriteCount',
                                    'videoTags', 'videoUniqueId', 'channelIdUnique', 'channelTitleCheck', 'channelDescription'], axis=1)
    
    # Identify and keep all duplicates
    duplicates = Target_File[Target_File.duplicated(keep=False)]  # Selects all duplicates, including the first occurrence
    
    # Remove duplicates
    Target_File = Target_File.drop_duplicates(ignore_index=True)
    
    # Filter for videos with 'videoDefaultAudioLanguage' starting with 'en'
    Target_File_EN = Target_File[Target_File['videoDefaultAudioLanguage'].str.startswith("en", na=False)].reset_index(drop=True)

    # Iterate through each row in 'Target_File_EN' to clean 'channelName' and 'videoTitle'
    for i in range(len(Target_File_EN['channelName'])):
        try:
            # Translate non-ASCII characters in 'channelName' and 'videoTitle' to English
            if not Target_File_EN['channelName'][i].isascii():
                Target_File_EN.loc[i, 'channelName'] = GoogleTranslator(source='auto', target='en').translate(Target_File_EN['channelName'][i])
            if not Target_File_EN['videoTitle'][i].isascii():
                Target_File_EN.loc[i, 'videoTitle'] = GoogleTranslator(source='auto', target='en').translate(Target_File_EN['videoTitle'][i])

            # Remove emojis
            Target_File_EN.loc[i, 'channelName'] = emoji.replace_emoji(Target_File_EN['channelName'][i], replace='')
            Target_File_EN.loc[i, 'videoTitle'] = emoji.replace_emoji(Target_File_EN['videoTitle'][i], replace='')

            # Decode HTML entities like &amp; and &#39;
            Target_File_EN.loc[i, 'channelName'] = html.unescape(Target_File_EN['channelName'][i])
            Target_File_EN.loc[i, 'videoTitle'] = html.unescape(Target_File_EN['videoTitle'][i])

            # Remove non-ASCII characters from 'channelName' and 'videoTitle'
            Target_File_EN.loc[i, 'channelName'] = re.sub(r'[^\x00-\x7F]+', '', Target_File_EN['channelName'][i])
            Target_File_EN.loc[i, 'videoTitle'] = re.sub(r'[^\x00-\x7F]+', '', Target_File_EN['videoTitle'][i])

        except Exception as e:
            print(e)
    
    # Remove duplicates after the transformations
    Target_File_EN = Target_File_EN.drop_duplicates(ignore_index=True)
    
    # Drop 'videoDefaultLanguage' column as it is no longer needed
    Target_File_EN = Target_File_EN.drop(['videoDefaultLanguage'], axis=1)
    
    # Fill missing values in 'channelCountry' with 'Unknown'
    Target_File_EN['channelCountry'] = Target_File_EN['channelCountry'].fillna('Unknown')
    
    return Target_File_EN

In [4]:
def videoDurationClassification(videoDurationInSeconds):
    """
    Classifies the video duration into categories based on its length in seconds.

    Args:
    - videoDurationInSeconds (int): Duration of the video in seconds.

    Returns:
    - str: A string indicating the classification of the video duration.
    """
    
    # Classifying the video duration into different categories
    if 0 <= videoDurationInSeconds <= 60:
        return "Very Short"  # Video duration between 0 and 60 seconds
    elif 61 <= videoDurationInSeconds <= 120:
        return "Short"  # Video duration between 61 and 120 seconds
    elif 121 <= videoDurationInSeconds <= 300:
        return "Medium"  # Video duration between 121 and 300 seconds (2-5 minutes)
    elif 301 <= videoDurationInSeconds <= 600:
        return "Long"  # Video duration between 301 and 600 seconds (5-10 minutes)
    elif 601 <= videoDurationInSeconds <= 3600:
        return "Very Long"  # Video duration between 601 seconds (10 minutes) and 3600 seconds (1 hour)
    elif 3601 <= videoDurationInSeconds <= 10800:
        return "Extended"  # Video duration between 3601 seconds (1 hour) and 10800 seconds (3 hours)
    elif videoDurationInSeconds > 10800:
        return "Ultra Long"  # Video duration greater than 10800 seconds (3 hours)
    else:
        return "Invalid video duration"  # Invalid value for video duration

# Channel Growth Calculation

## **Overview**
The channel growth metric is designed to assess the growth of a YouTube channel based on key engagement indicators: 
- **Channel View Count**
- **Channel Subscriber Count**
- **Channel Video Count**
- **Channel Age (in years)**

## **Formula**
The channel growth score is computed as:

```python
channel_growth = ((normalized_views * weight_views) + 
                  (normalized_subscribers * weight_subscribers) + 
                  (normalized_videos * weight_videos)) / channel_age_in_years
```

where:
- **Min-Max Normalization** is applied to views, subscribers, and video count:
  ```python
  normalized_value = (value - min_value) / (max_value - min_value)
  ```
- **Weighting factors** determine the relative importance of each metric:
  - `weight_views = 0.5`
  - `weight_subscribers = 0.3`
  - `weight_videos = 0.2`
- **Channel Age (years)** is computed from:
  ```python
  channel_age_in_years = (current_timestamp - channelPublishedOnInSeconds) / (365 * 24 * 60 * 60)
  ```

## **Concepts Used**
1. **Min-Max Normalization**: Scales values between 0 and 1.
   - [Feature Scaling (Wikipedia)](https://en.wikipedia.org/wiki/Feature_scaling)
2. **Weighted Scoring**: Prioritizes key metrics based on impact.
   - [Weighted Scoring Model](https://theproductmanager.com/topics/weighted-scoring-model/)
3. **YouTube Analytics Metrics**: Defines the importance of views, subscribers, and videos.
   - [YouTube Analytics Help](https://support.google.com/youtube/answer/9002587?hl=en)
4. **Channel Age Calculation**: Determines the time span since the channel was created.
   - [Unix Time Conversion](https://www.unixtimestamp.com/)

## **Implementation Example**
```python
import time
import datetime
import pytz

def calculate_channel_growth(view_count, subscriber_count, video_count, channel_published_on):
    utc_timestamp = int(time.time())
    zone = pytz.timezone('Asia/Kolkata')
    current_ist_time = datetime.datetime.fromtimestamp(utc_timestamp, zone)
    channel_age_in_years = (current_ist_time - channel_published_on) / (365 * 24 * 60 * 60)
    
    # Min-Max normalization (Assume predefined min/max values from dataset)
    min_views, max_views = 1000, 10000000
    min_subscribers, max_subscribers = 10, 1000000
    min_videos, max_videos = 1, 10000
    
    normalized_views = (view_count - min_views) / (max_views - min_views)
    normalized_subscribers = (subscriber_count - min_subscribers) / (max_subscribers - min_subscribers)
    normalized_videos = (video_count - min_videos) / (max_videos - min_videos)
    
    # Weighted sum
    growth_score = ((normalized_views * 0.5) + (normalized_subscribers * 0.3) + (normalized_videos * 0.2)) / channel_age_in_years
    
    return growth_score
```

## **Conclusion**
This approach helps in analyzing a YouTube channel's growth potential by factoring in **engagement, longevity, and content volume**. It provides a scalable and adaptable framework for evaluating growth trends over time.

In [5]:
def normalize(series):
    """
    Applies Min-Max normalization to a Pandas Series, scaling the values to a range between 0 and 1.

    Args:
    - series (pd.Series): The input data series to normalize.

    Returns:
    - pd.Series: A normalized series with values scaled between 0 and 1.
    """
    
    # Min-Max normalization formula: (x - min) / (max - min)
    return (series - series.min()) / (series.max() - series.min())

In [6]:
def parse_datetime(value):
    """
    Parses a datetime string into a Pandas datetime object based on specific formats.
    
    Handles two datetime formats:
    1. "%Y-%m-%dT%H:%M:%SZ" for ISO 8601 format without fractional seconds.
    2. "%Y-%m-%dT%H:%M:%S.%fZ" for ISO 8601 format with fractional seconds.

    Args:
    - value (str): The input datetime string to be parsed.

    Returns:
    - pd.Timestamp or pd.NaT: A Pandas Timestamp object if the format matches, otherwise pd.NaT.
    """
    
    # Check for the presence of "Z" and determine the format based on whether the string contains a decimal point
    if "Z" in value and "." not in value:
        return pd.to_datetime(value, format="%Y-%m-%dT%H:%M:%SZ")  # Format without fractional seconds
    elif "Z" in value and "." in value:
        return pd.to_datetime(value, format="%Y-%m-%dT%H:%M:%S.%fZ")  # Format with fractional seconds
    else:
        return pd.NaT  # Return Not a Time (NaT) if the format doesn't match

In [7]:
def calculate_channel_growth(Cleaned_File):
    """
    Calculates the growth score for channels and engagement score for videos based on various metrics.
    
    The growth score for each channel is calculated using the normalized values of:
    - View count
    - Subscriber count
    - Video count
    - Channel age (in years)
    
    The engagement score for each video is calculated using:
    - Views per day
    - Like-to-view ratio
    - Comment-to-view ratio

    Args:
    - Cleaned_File (pd.DataFrame): The dataframe containing channel and video data to calculate growth and engagement scores.

    Returns:
    - pd.DataFrame: The cleaned dataframe with the added columns for growth and engagement scores.
    """
    
    # Get the current IST time (Indian Standard Time) to calculate age-based metrics
    utc_timestamp = int(time.time())
    zone = pytz.timezone('Asia/Kolkata')
    current_ist_time = datetime.datetime.fromtimestamp(utc_timestamp, zone).replace(tzinfo=None) 
    
    # Channel Age Calculation (in years)
    channelPublishedOn = Cleaned_File["channelPublishedOn"].apply(parse_datetime)
    Cleaned_File['channelAgeInYears'] = (current_ist_time - channelPublishedOn).dt.total_seconds() / (365 * 24 * 60 * 60)
    
    # Min-Max normalization for channel metrics
    Cleaned_File['channelNormalizedViewCount'] = normalize(Cleaned_File['channelViewCount'])
    Cleaned_File['channelNormalizedSubscriberCount'] = normalize(Cleaned_File['channelSubscriberCount'])
    Cleaned_File['channelNormalizedVideoCount'] = normalize(Cleaned_File['channelVideoCount'])
    Cleaned_File['channelNormalizedChannelAge'] = normalize(Cleaned_File['channelAgeInYears'])
    
    # Define weights for each metric in the growth score calculation
    weight_views = 0.5
    weight_subscribers = 0.3
    weight_videos = 0.2
    
    # Growth Score Calculation for the channel
    Cleaned_File['channelGrowthScore'] = (
        (Cleaned_File['channelNormalizedViewCount'] * weight_views) +
        (Cleaned_File['channelNormalizedSubscriberCount'] * weight_subscribers) +
        (Cleaned_File['channelNormalizedVideoCount'] * weight_videos)
    ) / Cleaned_File['channelNormalizedChannelAge']
    
    # Video Age Calculation (in days)
    videoPublishedOn = Cleaned_File["videoPublishedOn"].apply(parse_datetime)
    Cleaned_File["videoAgeInDays"] = (current_ist_time - videoPublishedOn).dt.total_seconds() / (24 * 60 * 60)
    
    # Engagement Metrics for videos
    Cleaned_File["videoViewsPerDay"] = Cleaned_File["videoViewCount"] / Cleaned_File["videoAgeInDays"]
    Cleaned_File["videoLikeToViewRatio"] = Cleaned_File["videoLikeCount"] / Cleaned_File["videoViewCount"]
    Cleaned_File["videoCommentToViewRatio"] = Cleaned_File["videoCommentCount"] / Cleaned_File["videoViewCount"]
    
    # Engagement Score Calculation for the video
    Cleaned_File["videoEngagementScore"] = (
        (Cleaned_File["videoViewsPerDay"] * 0.5) +
        (Cleaned_File["videoLikeToViewRatio"] * 100 * 0.3) +
        (Cleaned_File["videoCommentToViewRatio"] * 100 * 0.2)
    )
    
    # Return the dataframe with added growth and engagement scores
    return Cleaned_File

In [8]:
def FeatureEngineering(Cleaned_File):
    """
    This function performs feature engineering to enhance the dataset for analysis by creating new features 
    and transforming existing ones, such as categorizing video duration, calculating channel growth and 
    video engagement scores, and enriching geographic details.

    The key steps include:
    - Extracting the day of the week from the video publish timestamp.
    - Classifying video durations into predefined categories.
    - Calculating channel growth and video engagement scores.
    - Ranking channels and videos based on their growth and engagement scores.
    - Merging geographic details like country, continent, and IT hub information with the dataset.
    
    Args:
    - Cleaned_File (pd.DataFrame): The input dataframe containing video and channel data for feature engineering.

    Returns:
    - pd.DataFrame: The transformed dataframe with newly engineered features.
    """

    # Feature: videoPublishedWeekDay - Derive the day of the week from the videoPublishedOn timestamp.
    Cleaned_File['videoPublishedWeekDay'] = pd.to_datetime(Cleaned_File["videoPublishedOn"]).dt.day_name()
    
    # Feature: videoDurationClassification - Categorize videos based on their duration in seconds into predefined segments.
    # Categories:
    #     Very Short (0 - 60 sec), Short (61 sec - 2 min), Medium (2 min 1 sec - 5 min),
    #     Long (5 min 1 sec - 10 min), Very Long (10 min 1 sec - 1 hour),
    #     Extended (1 hour 1 sec - 3 hours), Ultra Long (3 hours 1 sec and above)
    Cleaned_File['videoDurationClassification'] = Cleaned_File['videoDurationInSeconds'].apply(videoDurationClassification)
    
    # Feature: channelGrowth metric - Calculate channel growth using factors such as views, subscribers, video count, and age.
    # Normalization of key columns: channelPublishedOn, channelViewCount, channelSubscriberCount, and channelVideoCount
    Cleaned_File = calculate_channel_growth(Cleaned_File)
    
    # Feature: videoEngagementScore - Calculate the video engagement score using video views, likes, and comments.
    # Normalization of key columns: videoPublishedOn, videoViewCount, videoLikeCount, and videoCommentCount
    Cleaned_File = calculate_channel_growth(Cleaned_File)  # This also handles the video engagement scores
    
    # Feature: channelGrowthScoreRank - Rank channels based on their growth score.
    Cleaned_File["channelGrowthScoreRank"] = Cleaned_File["channelGrowthScore"].rank()
    
    # Feature: videoEngagementScoreRank - Rank videos based on their engagement score.
    Cleaned_File["videoEngagementScoreRank"] = Cleaned_File["videoEngagementScore"].rank()
    
    # Feature: Geographic Classification - Enrich dataset with geographic details (country, continent, IT hub classification).
    # This merges additional country and continent details from an external source based on the channel's country.
    
    # Fetch geographic details (ISO codes, country names, continent, etc.) from an external file
    Country_Details_ISO = Requirement_File_Extraction(repo_url, kaggle_repo_url, requirement_path).transpose()
    Country_Details_ISO = Country_Details_ISO.reset_index()
    Country_Details_ISO.rename(columns={'index': 'country_code'}, inplace=True)
    
    # Merge geographic details (from Country_Details_ISO) with the cleaned file
    resultDataFrame = pd.merge(Cleaned_File, Country_Details_ISO, left_on='channelCountry', right_on='country_code', how='left')
    
    # Fill missing geographic data with 'Unknown' (in case a country code doesn't match)
    resultDataFrame.fillna('Unknown', inplace=True)
    
    # Return the enriched dataframe with new features
    return resultDataFrame


In [9]:
def GitHubPush(Target_File_EN):
    """
    This function handles the process of saving a cleaned and processed DataFrame as a JSON file, 
    pushing it to a GitHub repository. It ensures that the file is properly named with a timestamp 
    and number of records, creates necessary directories, and commits the changes to the repository.
    
    Args:
    - Target_File_EN (pd.DataFrame): The DataFrame that contains the processed data to be saved and pushed.
    
    Returns:
    - None: This function performs file handling and Git operations but does not return anything.
    """

    # Count the number of records in the DataFrame
    record_count = len(Target_File_EN)
    
    # Generate a timestamp for the file name using the current time in IST (Indian Standard Time).
    timestamp = datetime.datetime.now(ist).strftime("%Y-%m-%d_%H:%M:%S")
    
    # Create a filename using the generated timestamp and number of records to ensure uniqueness.
    filename = f"DC_{timestamp}_{record_count}_records.json"
    
    # Save the DataFrame to a JSON file in a readable format (with indentation)
    Target_File_EN.to_json(filename, orient="records", indent=4)
    print(f"DataFrame saved as {filename}")
    
    # Check if the destination directory exists
    if not os.path.exists(destination_path):
        # If the directory does not exist, create it
        os.makedirs(destination_path)
        print('Created the destination directory, DataCleaning/Daily')
        # Copy the saved file into the newly created directory
        shutil.copyfile(f'/kaggle/working/{filename}', f'{destination_path}/{filename}')
    else:
        print('Destination directory already exists')
        # Copy the file to the existing directory
        shutil.copyfile(f'/kaggle/working/{filename}', f'{destination_path}/{filename}')
    
    # Initialize the repository for git operations using the local GitHub repository URL
    repo = Repo(kaggle_repo_url)
    
    # Add the copied file to the staging area for git commit
    repo.index.add([f"{destination_path}/{filename}"])
    
    # Create a timestamp for the commit message
    timestamp = datetime.datetime.now(ist).strftime("%Y-%m-%d_%H:%M:%S")
    # Commit the changes with a message that includes the timestamp and the filename
    repo.index.commit(f"{timestamp} Added files from Kaggle notebook, {filename}")
    
    # Push the changes to the remote repository
    origin = repo.remote(name="origin")
    push_result = origin.push()
    
    # Check if the push was successful and print the result
    if push_result:
        print("Push successful.")
    else:
        print("Push failed.")

In [10]:
def main():
    """
    The main function orchestrates the entire data pipeline by:
    1. Extracting the source data from the given repository URL.
    2. Cleaning the extracted data using the DataCleaning function.
    3. Applying feature engineering to the cleaned data using the FeatureEngineering function.
    4. Pushing the final processed file to a GitHub repository.
    
    This function executes the steps in sequence to process and upload data.
    
    Args:
    - None: This function does not accept any arguments. It uses predefined repository URLs and paths.
    
    Returns:
    - None: This function does not return anything but performs data processing and Git operations.
    """
    
    # Step 1: Extract the source file from the repository based on the provided URL and path.
    Source_File = Source_File_Extraction(repo_url, kaggle_repo_url, source_path)
    
    # Step 2: Clean the extracted data using the DataCleaning function.
    Cleaned_File = DataCleaning(Source_File)
    
    # Optional: Uncomment to display the cleaned file sorted by video duration.
    # display(Cleaned_File.sort_values(by='videoDurationInSeconds', ascending=True))
    
    # Step 3: Apply feature engineering to the cleaned data.
    Feature_File = FeatureEngineering(Cleaned_File)
    
    # Optional: Uncomment to display the feature-engineered file.
    # display(Feature_File)
    
    # Step 4: Push the processed and feature-engineered data to GitHub using GitHubPush function.
    GitHubPush(Feature_File)

In [11]:
if __name__ == "__main__":
    """
    This script is the entry point for the data cleaning and feature engineering pipeline.
    It performs the following tasks:
    1. Imports necessary libraries for data processing, file handling, and Git operations.
    2. Retrieves user secrets for repository URL.
    3. Sets up paths for different directories (source, destination, etc.).
    4. Configures pandas to display all columns and rows without truncation.
    5. Calls the main function to execute the pipeline.

    The script is designed to be executed as the main module in a Python environment.
    It ensures that all necessary operations are performed, including fetching source data, 
    cleaning, feature engineering, and pushing the final data to a GitHub repository.
    """

    # Import necessary libraries
    import os  
    import git  # Git library for interacting with repositories
    from git import Repo  # GitHub repository interaction
    import time  # For time-related operations
    import datetime  # For working with date and time
    from pytz import timezone  # For timezone management
    import pytz  # Timezone handling
    import pandas as pd  # For data manipulation and analysis
    import deep_translator  # For translation services
    from deep_translator import GoogleTranslator  # Google Translate API integration
    import shutil  # For file operations like copying or removing
    import emoji  # For handling emojis in the data
    import re  # For regular expression operations
    import html  # For HTML parsing and escaping
    from kaggle_secrets import UserSecretsClient  # For accessing Kaggle's secret management system
    
    # Retrieve secret value for repository URL from Kaggle secrets storage
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("dataCleanRepoUrl")
    repo_url = secret_value_0  # URL for the GitHub repository used in this pipeline
    
    # Set timezone to Indian Standard Time (IST)
    ist = timezone('Asia/Kolkata')
    
    # Define paths for different directories
    kaggle_repo_url = '/kaggle/working/YouTubeFoodChannelAnalysis'  # Path to the working repository on Kaggle
    destination_path = '/kaggle/working/YouTubeFoodChannelAnalysis/DataCleaning/Daily'  # Path to store cleaned data
    source_path = '/kaggle/working/YouTubeFoodChannelAnalysis/Source/Daily'  # Path to source raw data
    requirement_path = '/kaggle/working/YouTubeFoodChannelAnalysis/Requirement/Daily'  # Path to requirement files
    
    # Configure pandas to display all columns and rows without truncation for easier debugging
    pd.set_option("display.max_columns", None)  # Prevent truncating columns
    pd.set_option("display.max_rows", None)  # Prevent truncating rows
    
    # Call the main function to execute the data pipeline
    main()

Successfully cloned the git repo
Already cloned and the repo file exists
Successfully pulled the git repo before push
DataFrame saved as DC_2025-02-05_03:56:28_412_records.json
Destination directory already exists
Push successful.
